## Extract Poses from Amass Dataset

In [32]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import zipfile
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm



from human_body_prior.tools.omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Please remember to download the following subdataset from AMASS website: https://amass.is.tue.mpg.de/download.php. Note only download the <u>SMPL+H G</u> data.
* ACCD (ACCD)
* HDM05 (MPI_HDM05)
* TCDHands (TCD_handMocap)
* SFU (SFU)
* BMLmovi (BMLmovi)
* CMU (CMU)
* Mosh (MPI_mosh)
* EKUT (EKUT)
* KIT  (KIT)
* Eyes_Janpan_Dataset (Eyes_Janpan_Dataset)
* BMLhandball (BMLhandball)
* Transitions (Transitions_mocap)
* PosePrior (MPI_Limits)
* HumanEva (HumanEva)
* SSM (SSM_synced)
* DFaust (DFaust_67)
* TotalCapture (TotalCapture)
* BMLrub (BioMotionLab_NTroje)

### Unzip all datasets. In the bracket we give the name of the unzipped file folder. Please correct yours to the given names if they are not the same.

### Place all files under the directory **./amass_data/**. The directory structure shoud look like the following:  
./amass_data/  
./amass_data/ACCAD/  
./amass_data/BioMotionLab_NTroje/  
./amass_data/BMLhandball/  
./amass_data/BMLmovi/   
./amass_data/CMU/  
./amass_data/DFaust_67/  
./amass_data/EKUT/  
./amass_data/Eyes_Japan_Dataset/  
./amass_data/HumanEva/  
./amass_data/KIT/  
./amass_data/MPI_HDM05/  
./amass_data/MPI_Limits/  
./amass_data/MPI_mosh/  
./amass_data/SFU/  
./amass_data/SSM_synced/  
./amass_data/TCD_handMocap/  
./amass_data/TotalCapture/  
./amass_data/Transitions_mocap/  

**Please make sure the file path are correct, otherwise it can not succeed.**

In [33]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [34]:
from human_body_prior.body_model.body_model import BodyModel

neutral_bm_path = 'body_models/smplx/SMPLX_NEUTRAL_2020.npz'
num_betas = 16
neutral_bm = BodyModel(bm_fname=neutral_bm_path, num_betas=num_betas, num_expressions=0).to(comp_device)
faces = c2c(neutral_bm.f)

In [35]:
import pathlib
from collections import defaultdict

amass_data_d = pathlib.Path('amass_data')

data_fs = [pathlib.Path(f'{rt}/{f}')
           for rt,ds,fs in os.walk(amass_data_d)
           for f in fs
           if f.endswith('.npz')]
data_fs.sort()

new_data_fs = list()
files_not_readable = list()
files_model_type_not_usable = list()
files_mocap_frame_rate_key_missing = list()
files_frame_rate_not_multiple_of_30 = list()
all_model_types = defaultdict(int)
all_fps = defaultdict(int)
for f in tqdm(data_fs,ncols=150):
    try:
        data = np.load(f,allow_pickle=True)
    except zipfile.BadZipFile:
        files_not_readable.append(f)
        continue
    all_model_types[data['surface_model_type'].item()] += 1
    if data['surface_model_type'].item() not in {'smplx','smplx_locked_head'}:
        files_model_type_not_usable.append(f)
        continue
    if 'mocap_frame_rate' not in data:
        files_mocap_frame_rate_key_missing.append(f)
        continue
    fps = int(data['mocap_frame_rate'].item())
    all_fps[fps] += 1
    if fps % 30 != 0:
        files_frame_rate_not_multiple_of_30.append(f)
        continue
    new_data_fs.append(f)
print('total files:',len(data_fs))
print('usable_files:',len(new_data_fs))
print('files not readable:',len(files_not_readable))
print('files model type not usable:',len(files_model_type_not_usable))
print('files missing mocap_frame_rate:',len(files_mocap_frame_rate_key_missing))
print('files with frame rante not multiple of 30fps:',len(files_frame_rate_not_multiple_of_30))
print('model types:',*sorted(all_model_types.items()))
print('mocap frame rate:',*sorted(all_fps.items()))
assert len(data_fs) == (len(new_data_fs) + 
                        len(files_not_readable) + 
                        len(files_model_type_not_usable) + 
                        len(files_mocap_frame_rate_key_missing) + 
                        len(files_frame_rate_not_multiple_of_30))
data_fs = new_data_fs

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 17430/17430 [00:28<00:00, 618.52it/s]

total files: 17430
usable_files: 9977
files not readable: 1
files model type not usable: 0
files missing mocap_frame_rate: 462
files with frame rante not multiple of 30fps: 6990
model types: ('smplx', 17429)
mocap frame rate: (59, 13) (60, 532) (100, 6920) (120, 9436) (150, 9) (250, 57)


In [38]:
trans_matrix = np.array([[1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0],
                         [0.0, 1.0, 0.0]])
ex_fps = 30
vis = 4
def amass_to_pose(motion_f, joints_f):
    bdata = np.load(motion_f, allow_pickle=True)
    fps = bdata['mocap_frame_rate']
    assert int(fps) % ex_fps == 0
    assert bdata['surface_model_type'].item() == 'smplx'
    assert bdata['gender'] == 'neutral'
    bm = neutral_bm
    down_sample = int(fps / ex_fps)
    bdata_poses = bdata['poses'][::down_sample,...]
    bdata_trans = bdata['trans'][::down_sample,...]
    body_parms = {
            'root_orient': torch.Tensor(bdata_poses[:, :3]).to(comp_device),
            'pose_body': torch.Tensor(bdata_poses[:, 3:66]).to(comp_device),
            'pose_hand': torch.Tensor(bdata_poses[:, 75:]).to(comp_device),
            'trans': torch.Tensor(bdata_trans).to(comp_device),
            'betas': torch.Tensor(np.repeat(bdata['betas'][:num_betas][np.newaxis], repeats=len(bdata_trans), axis=0)).to(comp_device),
        }
    with torch.no_grad():
        body = bm(**body_parms)
    pose_seq_np = body.Jtr.detach().cpu().numpy()
    # Make XZ plane the ground plane.
    pose_seq_np_n = np.dot(pose_seq_np, trans_matrix)    
    np.save(joints_f,pose_seq_np_n)

    ############################################################################
    global vis
    if vis > 0:
        import trimesh
        vertices_all = body.v.detach().cpu().numpy()
        faces = body.f.detach().cpu().numpy()
        src_f = pathlib.Path(motion_f).relative_to('amass_data')
        ply_d = (pathlib.Path('/vision/vision_data_2/VGGSound_shards_fixed/shrinidhi/meshes_amass_pose_data') / 
                src_f.parent / src_f.stem)
        ply_d.mkdir(parents=True,exist_ok=True)
        for f,vertices in enumerate(vertices_all):
            obj_f = ply_d / f'{f:05}.ply'
            mesh = trimesh.Trimesh(vertices=vertices,
                                faces=faces,
                                process=False)
            mesh.export(obj_f)
        vis -= 1
    ############################################################################

In [39]:
pose_data_d = pathlib.Path('amass_pose_data')
for f in tqdm(data_fs,desc='amass-to-pose',ncols=150):
    out_f = pose_data_d / f.relative_to(amass_data_d).with_suffix('.npy')
    out_f.parent.mkdir(parents=True,exist_ok=True)
    if out_f.is_file():
        out_f.unlink()
    amass_to_pose(f,out_f)

amass-to-pose:   0%|                                                                                                         | 0/9977 [00:00<?, ?it/s]

amass-to-pose: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 9977/9977 [05:40<00:00, 29.32it/s]


## Segment, Mirror and Relocate Motions

In [11]:
from collections import defaultdict
import csv
import os
import pathlib

from tqdm import tqdm
import numpy as np

bm_params_f = pathlib.Path('body_models/smplx/SMPLX_NEUTRAL_2020.npz')
index_f = pathlib.Path('index.csv')
pose_data_d  = pathlib.Path('pose_data_amass')
joints_d = pathlib.Path('joints_amass')

Find the corresponding left/right joints from model npy file. We will mirror left/right joints to augment data.

In [13]:
bm_params = np.load(bm_params_f,allow_pickle=True)
joint2ind = bm_params['joint2num'].item()
ind2joint = {v:k
             for k,v in joint2ind.items()}
l_joints,r_joints = list(),list()
for j in joint2ind:
    if j.startswith('L_'):
        l_j = j
        r_j = j.replace('L_','R_')
        l_joints.append(joint2ind[l_j])
        r_joints.append(joint2ind[r_j])
joints_to_drop = [joint2ind['Jaw'],
                  joint2ind['L_Eye'],
                  joint2ind['R_Eye']]

print('num joints to swap:',len(l_joints))
print('left joints:',l_joints)
print('right joints:',r_joints)
print('joints to drop:',joints_to_drop)
for l,r in sorted(zip(l_joints,r_joints)):
    print(f'{ind2joint[l]:10} ({l:2}) <--> ({r:2}) {ind2joint[r]}')

num joints to swap: 24
left joints: [30, 23, 13, 34, 35, 36, 39, 38, 37, 27, 26, 25, 10, 4, 18, 31, 32, 33, 28, 29, 20, 16, 1, 7]
right joints: [45, 24, 14, 49, 50, 51, 54, 53, 52, 42, 41, 40, 11, 5, 19, 46, 47, 48, 43, 44, 21, 17, 2, 8]
joints to drop: [22, 23, 24]
L_Hip      ( 1) <--> ( 2) R_Hip
L_Knee     ( 4) <--> ( 5) R_Knee
L_Ankle    ( 7) <--> ( 8) R_Ankle
L_Foot     (10) <--> (11) R_Foot
L_Collar   (13) <--> (14) R_Collar
L_Shoulder (16) <--> (17) R_Shoulder
L_Elbow    (18) <--> (19) R_Elbow
L_Wrist    (20) <--> (21) R_Wrist
L_Eye      (23) <--> (24) R_Eye
L_Index1   (25) <--> (40) R_Index1
L_Index2   (26) <--> (41) R_Index2
L_Index3   (27) <--> (42) R_Index3
L_Middle1  (28) <--> (43) R_Middle1
L_Middle2  (29) <--> (44) R_Middle2
L_Middle3  (30) <--> (45) R_Middle3
L_Pinky1   (31) <--> (46) R_Pinky1
L_Pinky2   (32) <--> (47) R_Pinky2
L_Pinky3   (33) <--> (48) R_Pinky3
L_Ring1    (34) <--> (49) R_Ring1
L_Ring2    (35) <--> (50) R_Ring2
L_Ring3    (36) <--> (51) R_Ring3
L_Thumb1 

To sample frames according to HumanML3D, create the dictionary of files to sample, their start/end frames, their original ids.  
Some sub-datasets of AMASS are not used by HumanML3D because these sub-datasets were added AMASS after HumanML3D was released. Refer to [this issue](https://github.com/EricGuo5513/HumanML3D/issues/96).

In [17]:
to_sample = defaultdict(list)
n_dropped = 0
n_missing = 0
for row in csv.DictReader(open(index_f)):
    data_f = row['source_path']
    data_f = data_f.replace('pose_data','pose_data_amass')
    # Not using `humanact12` dataset. Discard those entries.
    if 'humanact12' in data_f:
        n_dropped += 1
        continue
    # SMPL-X version of AMASS is missing the following. Discard those entries.
    if ('BMLhandball' in data_f or
        'DanceDB' in data_f or
        'HUMAN4D' in data_f or 
        'CMU/22_23_Rory' in data_f or
        'CMU/18_19_rory' in data_f or
        'CMU/18_19_Justin' in data_f or
        'CMU/20_21_rory1' in data_f or
        'CMU/20_21_Justin1' in data_f or
        'CMU/22_23_justin' in data_f):
        n_dropped += 1
        continue
    # SMPL-X version of AMASS has renamed many files. Map the old names to new.
    data_f = (data_f
              .replace('/BioMotionLab_NTroje/','/BMLrub/')
              .replace('/DFaust_67/','/DFaust/')
              .replace('/MPI_mosh/','/MoSh/')
              .replace('/MPI_HDM05/','/HDM05/')
              .replace('/MPI_Limits/','/PosePrior/')
              .replace('/SSM_synced/','/SSM/')
              .replace('/TCD_handMocap/','/TCDHands/')
              .replace('/Transitions_mocap/','/Transitions/')
              .replace('.npz','')
              .replace('.npy','')
              .replace('_poses','')
              .replace(' ','_'))
    data_f = pathlib.Path(data_f)
    data_d = data_f.parent
    if not data_d.is_dir():
        n_missing += 1
        continue
    for f in data_d.iterdir():
        if f.name.startswith(data_f.name):
            to_sample[f].append((int(row['start_frame']),
                                 int(row['end_frame']),
                                 row['new_name'].replace('.npy','')))
            break
    else:
        # assert False
        n_missing += 1
print('missing files:',n_missing)

missing files: 5192


Many files need to be sampled multiple times because there are multiple entries in `index.csv` for those files.

In [18]:
print('samples dropped:',n_dropped)
print('samples missing:',n_missing)
print('files to sample:',len(to_sample))
print('files with >1 sample:', 
      sum(1
          for v in to_sample.values()
          if len(v) > 1))
print('total samples:',
      sum(len(v)
          for v in to_sample.values()))

samples dropped: 1365
samples missing: 5192
files to sample: 5498
files with >1 sample: 2067
total samples: 8059


Gather all file names.

In [19]:
data_fs = [pathlib.Path(f'{rt}/{f}')
           for rt,ds,fs in os.walk(pose_data_d)
           for f in fs
           if f.endswith('.npy')]
data_fs.sort()
print('num files:',len(data_fs))

num files: 9977


Sample each file as per HumanML3D, create a mirrored version of the sample, save both.  
Following code is explained in [this issue](https://github.com/EricGuo5513/HumanML3D/issues/20).
```
data[...,0] *= -1
```

In [20]:
n_dropped = 0
pose_data_to_joints_map = ['file,new_id,orig_id']
pbar = tqdm(data_fs,desc='mirror & prune files',ncols=150)
for i,f in enumerate(pbar):
    if f not in to_sample:
        n_dropped += 1
        continue
    for j,(i_beg,i_end,orig_id) in enumerate(sorted(to_sample[f])):
        id = f'{i:06}_{j:02}'
        id_m = f'M{i:06}_{j:02}'
        out_f = joints_d / f'{id}.npy'
        out_m_f = joints_d / f'{id_m}.npy'
        if out_f.is_file():
            out_f.unlink()
        if out_m_f.is_file():
            out_m_f.unlink()
        data = np.load(f)
        if 'humanact12' not in str(f):
            if 'Eyes_Japan_Dataset' in str(f):
                data = data[3*fps:]
            if 'HDM05' in str(f):
                data = data[3*fps:]
            if 'TotalCapture' in str(f):
                data = data[1*fps:]
            if 'PosePrior' in str(f):
                data = data[1*fps:]
            if 'Transitions' in str(f):
                data = data[int(0.5*fps):]
        data = data[i_beg:i_end]
        data_m = data.copy()
        data_m[:,l_joints] = data[:,r_joints]
        data_m[:,r_joints] = data[:,l_joints]
        data[...,0] *= -1
        data = np.delete(data,joints_to_drop,axis=1)
        data_m = np.delete(data_m,joints_to_drop,axis=1)
        np.save(out_f,data)
        np.save(out_m_f,data_m)
        pose_data_to_joints_map.append(f'{f},{id},{orig_id}')
        pbar.set_postfix({'samples':2*len(pose_data_to_joints_map),
                          'dropped':n_dropped})
_ = open('pose_data_to_joints_map_amass.txt','w').write('\n'.join(pose_data_to_joints_map) + '\n')
pbar.close()

mirror & prune files: 100%|██████████████████████████████████████████████████████████| 9977/9977 [02:07<00:00, 78.02it/s, samples=16120, dropped=4478]


Checkout original train/val/test sets and texts. Just to make sure we have the original versions.

In [23]:
!git checkout main -- HumanML3D/train.txt HumanML3D/val.txt HumanML3D/test.txt HumanML3D/texts.zip
!rm -rf HumanML3D/train_orig.txt HumanML3D/val_orig.txt HumanML3D/test_orig.txt HumanML3D/texts_orig HumanML3D/texts
!unzip -q HumanML3D/texts.zip -d HumanML3D
!rm HumanML3D/texts.zip
!mv HumanML3D/train.txt HumanML3D_amass/train_orig.txt
!mv HumanML3D/val.txt HumanML3D_amass/val_orig.txt
!mv HumanML3D/test.txt HumanML3D_amass/test_orig.txt
!mv HumanML3D/texts HumanML3D_amass/texts_orig


EnvironmentNameNotFound: Could not find conda environment: avjoint
You can list all discoverable environments with `conda info --envs`.



EnvironmentNameNotFound: Could not find conda environment: avjoint
You can list all discoverable environments with `conda info --envs`.



EnvironmentNameNotFound: Could not find conda environment: avjoint
You can list all discoverable environments with `conda info --envs`.



EnvironmentNameNotFound: Could not find conda environment: avjoint
You can list all discoverable environments with `conda info --envs`.



EnvironmentNameNotFound: Could not find conda environment: avjoint
You can list all discoverable environments with `conda info --envs`.



EnvironmentNameNotFound: Could not find conda environment: avjoint
You can list all discoverable environments with `conda info --envs`.



EnvironmentNameNotFound: Could not find conda environment: avjoint
You can list all discoverable environments with `conda info --envs`.



EnvironmentNameNotFound: C

Make new test set from original test set with missing samples removed. SMPL-X version of AMASS doesn't seem to have all the original samples from SMPL-H version.

In [24]:
orig_test = [l.strip()
             for l in open('HumanML3D_amass/test_orig.txt')]
print('orig test set:',len(orig_test))

pose_data_to_joints_map_f = pathlib.Path('pose_data_to_joints_map_amass.txt')
orig_id_to_new_id = {row['orig_id']:row['new_id']
                     for row in csv.DictReader(open(pose_data_to_joints_map_f))}

index_f = pathlib.Path('index.csv')
orig_id_to_orig_file = {row['new_name'].replace('.npy',''):row['source_path']
                        for row in csv.DictReader(open(index_f))}

new_test = list()
for orig_id in orig_test:
    is_m_id = orig_id.startswith('M')
    if is_m_id:
        orig_m_id = orig_id
        orig_id = orig_m_id[1:]
    else:
        orig_m_id = f'M{orig_id}'
    data_f = orig_id_to_orig_file[orig_id]
    # Not using `humanact12` dataset. Discard those entries in val set.
    if 'humanact12' in data_f:
        continue
    # SMPL-X version of AMASS is missing the following. Ignore them.
    if ('BMLhandball' in data_f or
        'DanceDB' in data_f or
        'HUMAN4D' in data_f or 
        'CMU/22_23_Rory' in data_f or
        'CMU/18_19_rory' in data_f or
        'CMU/18_19_Justin' in data_f or
        'CMU/20_21_rory1' in data_f or
        'CMU/20_21_Justin1' in data_f or
        'CMU/22_23_justin' in data_f):
        continue
    if orig_id not in orig_id_to_new_id:
        continue
    new_test.append(orig_id_to_new_id[orig_id])
open('HumanML3D_amass/test.txt','w').write('\n'.join(sorted(new_test))+'\n')
print('new test set:',len(new_test))

orig test set: 4384
new test set: 2460


Use the remaining samples as the train set.

In [26]:
orig_train = [l.strip()
             for l in open('./HumanML3D_amass/train_orig.txt')]
print('orig train set:',len(orig_train))

new_test = set(new_test)
joints_d = pathlib.Path('joints_amass')
new_train = list()
for f in tqdm(list(sorted(joints_d.iterdir())),desc='sample',ncols=150):
    id = f.with_suffix('').name
    if id not in new_test:
        new_train.append(id)
_ = open('HumanML3D_amass/train.txt','w').write('\n'.join(sorted(new_train)) + '\n')
print('new train set:',len(new_train))

orig train set: 23384


sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 16118/16118 [00:00<00:00, 375747.80it/s]

new train set: 14888


In [27]:
!rm HumanML3D_amass/train_orig.txt HumanML3D_amass/val_orig.txt HumanML3D_amass/test_orig.txt


EnvironmentNameNotFound: Could not find conda environment: avjoint
You can list all discoverable environments with `conda info --envs`.




Copy the corresponding text file for each example in the new train/test set.

In [28]:
import shutil
import csv

texts_orig_d = pathlib.Path('HumanML3D_amass/texts_orig')
texts_d = pathlib.Path('HumanML3D_amass/texts')
texts_d.mkdir(parents=True,exist_ok=True)

pose_data_to_joints_map_f = pathlib.Path('pose_data_to_joints_map_amass.txt')
new_id_to_orig_id = {row['new_id']:row['orig_id']
                     for row in csv.DictReader(open(pose_data_to_joints_map_f))}
for f in tqdm(list(sorted(joints_d.iterdir())),desc='texts',ncols=150):
    id = f.with_suffix('').name
    text_f = texts_d / f'{id}.txt'
    if id.startswith('M'):
        id = id[1:]
    orig_id = new_id_to_orig_id[id]
    text_orig_f = texts_orig_d / f'{orig_id}.txt'
    shutil.copyfile(text_orig_f,text_f)

texts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 16118/16118 [02:30<00:00, 106.83it/s]


In [29]:
!rm -r HumanML3D_amass/texts_orig


EnvironmentNameNotFound: Could not find conda environment: avjoint
You can list all discoverable environments with `conda info --envs`.


